In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("../..")

In [3]:
TEST_PERCENTAGE = 0.2
VAL_PERCENTAGE = 0.2

VAL_TRAIN_RATIO = VAL_PERCENTAGE / (1 - TEST_PERCENTAGE)

SEED = 0xCAFFE

In [4]:
import pandas as pd
import numpy as np
import pickle as pkl

In [5]:
from tweet_recommendations.data_processing.split_train_test import get_train_test_users

In [6]:
municipal_users = pd.read_pickle("data/our/users.pkl")
municipal_tweets = pd.read_pickle("data/our/tweets.pkl")

In [7]:
municipal_splits = {}
for splitter in ['retweet_count', 'followers']:

    train_val_ids, test_ids = get_train_test_users(municipal_tweets, municipal_users, 
                                                   split_on=splitter, 
                                                   test_size=TEST_PERCENTAGE, 
                                                   random_state=0xCAFFE)
    twdf = municipal_tweets[municipal_tweets['user_id'].isin(train_val_ids)]
    train_ids, val_ids = get_train_test_users(twdf, municipal_users, split_on=splitter, 
                                              test_size=VAL_TRAIN_RATIO, 
                                              random_state=0xCAFFE)
    
    municipal_splits[splitter] = {"train_user_ids": train_ids.tolist(), 
                                  "val_user_ids": val_ids.tolist(),
                                  "test_user_ids": test_ids.tolist()}

In [8]:
with open("./data/our/train_val_test_splits.pkl", "wb") as f:
    pkl.dump(municipal_splits, f)